In [4]:
# Import Library

import pandas as pd

In [5]:
# Load Data
yjmob1 = 'task1_dataset.csv.gz'
yjmob_df = pd.read_csv(yjmob1, compression='gzip')
full_yjmob_df = pd.read_csv('yjmob100k-dataset1.csv.gz', compression='gzip')

# Load All User Ids
uids = sorted(yjmob_df['uid'].unique())

print("Finished Loading Raw Data!")

# Transforming (x,y) location to grid-id
def generate_spatial_token(x, y):
    return (x-1)+(y-1)*200

# Assign (x,y) location to a corresponding id
yjmob_df = yjmob_df.copy()
yjmob_df['combined_xy'] = yjmob_df.apply(lambda row: generate_spatial_token(row['x'], row['y']), axis=1)

print("Finished generated location ID for avilable each (x,y) coordinate!")

Finished Loading Raw Data!
Finished generated location ID for avilable each (x,y) coordinate!


In [14]:
# Extract test user ids
test_uids  = list(yjmob_df[yjmob_df['x']==999]['uid'].unique())

# Extract train user ids
train_uids = list(yjmob_df[~yjmob_df['uid'].isin(test_uids)]['uid'].unique())

In [16]:
# Extract Test Data
df_test = yjmob_df[yjmob_df['uid'].isin(test_uids)].reset_index(drop=True)
# Result the Unknown Location grid-id to 999
df_test.loc[df_test['combined_xy']==200598,'combined_xy']=999

# Extract Train Data
df_train = yjmob_df[yjmob_df['uid'].isin(train_uids)].reset_index(drop=True)

In [17]:
# Formatting Step: for all data, I disregard any data that is outside of the (48*4+48) window for N number of windows
# for 48*4 being the input size and 48 being the output size of a predictive model

list_train = [] # record all train data
one_traj = 192+48

# Extract all train data
for uid, group in df_train.groupby('uid'):
    # group = group.reset_index()
    num_one_traj = int(len(group) / one_traj)
    total_one_traj = num_one_traj * one_traj
    list_train.append(group.head(total_one_traj))

# Extract all available test data and use them as train data
list_test = [] # record the available test data
for uid, group in df_test.groupby('uid'):
    group = group[group['combined_xy']!=999]
    num_one_traj = int(len(group) / one_traj)
    total_one_traj = num_one_traj * one_traj
    list_test.append(group.head(total_one_traj)) # load available test data
    list_train.append(group.head(total_one_traj)) # input available test data to train dataset

df_train = pd.concat(list_train)[['uid', 't', 'combined_xy']]

In [18]:
# Extract unknown test data 
list_true_test = []
for uid, group in df_test.groupby('uid'):
    group = group[group['combined_xy']==999]   # getting all the predicted data
    first_day_to_test = group['d'].tolist()[0] # getting all the predicted data of the first day I need to make prediction
    group = group[group['d']==first_day_to_test]
    list_true_test.append(group)
df_true_test = pd.concat(list_true_test)[['uid', 'd', 't']] # should be predicting only the first day steps

In [19]:
# Extract Ground Truth
full_test_yjmob_df = full_yjmob_df[full_yjmob_df['uid'].isin(test_uids)]
df_true_test = df_true_test.merge(full_test_yjmob_df, on=['uid', 'd', 't'])[['uid', 't', 'x', 'y']]

# Also extract the available test data for autoregressive prediction
df_test = pd.concat(list_test)[['uid', 't', 'combined_xy']]

In [21]:
# Convert to csv

df_train.to_csv('train.csv', index=False) # training data (train + available test)
df_test.to_csv('test.csv', index=False) # available test data
df_true_test.to_csv('true_test.csv', index=False) # ground truth